# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#512 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846770632607                   -0.70    4.0
  2   -7.852314255994       -2.26       -1.53    1.0
  3   -7.852646043452       -3.48       -2.52    3.2
  4   -7.852646677507       -6.20       -3.34    2.2
  5   -7.852646685954       -8.07       -4.79    1.2
  6   -7.852646686726       -9.11       -5.25    4.0
  7   -7.852646686730      -11.49       -5.97    1.8
  8   -7.852646686730      -12.59       -7.42    2.0
  9   -7.852646686730   +    -Inf       -7.68    2.5
 10   -7.852646686730   +  -14.75       -8.80    2.0
 11   -7.852646686730      -14.57       -9.82    3.2
 12   -7.852646686730   +  -15.05      -10.38    1.8
 13   -7.852646686730      -15.05      -11.38    2.0
 14   -7.852646686730   +  -14.35      -11.70    1.8
 15   -7.852646686730      -14.45      -12.56    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846792967547                   -0.70           4.0
  2   -7.852526702003       -2.24       -1.62   0.80    5.0
  3   -7.852610431717       -4.08       -2.79   0.80    5.2
  4   -7.852646564723       -4.44       -3.57   0.80    7.2
  5   -7.852646683743       -6.92       -4.56   0.80    8.8
  6   -7.852646686682       -8.53       -5.14   0.80   10.5
  7   -7.852646686725      -10.36       -6.21   0.80   11.8
  8   -7.852646686730      -11.35       -7.22   0.80   13.5
  9   -7.852646686730      -13.71       -7.98   0.80   15.8
 10   -7.852646686730      -15.05       -8.66   0.80   16.8
 11   -7.852646686730   +  -15.05       -9.81   0.80   17.5
 12   -7.852646686730      -14.75      -10.46   0.80   19.5
 13   -7.852646686730      -15.05      -11.35   0.80   21.0
 14   -7.852646686730   +  -14.57      -11.99   0.80   23.5
 15   -7.852646686730      -15.05      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.414257e+01     3.317151e+00
 * time: 0.5633599758148193
     1     9.927926e-01     1.790464e+00
 * time: 0.8228960037231445
     2    -2.159415e+00     2.185110e+00
 * time: 0.853186845779419
     3    -4.224619e+00     1.832310e+00
 * time: 1.0284018516540527
     4    -5.770634e+00     1.757968e+00
 * time: 1.074474811553955
     5    -7.197671e+00     9.103321e-01
 * time: 1.1179308891296387
     6    -7.480870e+00     1.179291e+00
 * time: 1.1493380069732666
     7    -7.688146e+00     8.187958e-01
 * time: 1.1798629760742188
     8    -7.741314e+00     9.427868e-01
 * time: 1.2110509872436523
     9    -7.775788e+00     6.555468e-01
 * time: 1.2415709495544434
    10    -7.790048e+00     7.539846e-01
 * time: 1.2718257904052734
    11    -7.812617e+00     3.231678e-01
 * time: 1.3019349575042725
    12    -7.825367e+00     1.574029e-01
 * time: 1.3334438800811768
    13    -7.834558e+00     6.894273e-02
 * time: 1.365992784500

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846824619695                   -0.70    3.8
  2   -7.852311071828       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686632                   -2.54
  2   -7.852646686730      -10.01       -5.87
  3   -7.852646686730   +  -14.45      -11.96
  4   -7.852646686730   +    -Inf      -15.77
|ρ_newton - ρ_scf|  = 9.810083248414307e-14
|ρ_newton - ρ_scfv| = 1.4716386486457013e-14
|ρ_newton - ρ_dm|   = 2.9980817188053747e-10
